In [15]:
autodiff(n::Number) = 0
autodiff(s::Symbol) = 1

autodiff (generic function with 4 methods)

In [55]:
function autodiff(::Type{Val{:+}}, ex ::Expr) :: Expr
    reduce((x, y) -> :($x + $y) ,map(autodiff, ex.args[2:end]))
end    
function autodiff(::Type{Val{:-}}, ex ::Expr) ::Expr
    reduce((x, y) -> :($x - $y) ,map(autodiff, ex.args[2:end]))
end    
function autodiff(::Type{Val{:*}}, ex ::Expr) ::Expr
    res = :(0)
    children = ex.args[2:end]
    diffs = map(autodiff, children)
    
    for i in 1:length(diffs)
        part = 1
        for j in 1:length(diffs)
            if i == j
                d = diffs[j]
                part = :($part * $d)
            else
                c = children[j]
                part = :($part * $c)
            end
        end
        res = :($res + $part)
    end
       
    res
end    
function autodiff(::Type{Val{:/}}, ex ::Expr) ::Expr
    f = ex.args[2]
    g = ex.args[3]
    df = autodiff(ex.args[2])
    dg = autodiff(ex.args[3])
    :(($df * $g - $f * $dg) / ($g * $g))
end    

autodiff (generic function with 4 methods)

In [56]:
function autodiff(ex::Expr)::Expr
    if (ex.head == :call) autodiff(Val{ex.args[1]}, ex) end
end    

autodiff (generic function with 5 methods)

In [66]:
autodiff(:(x * x / 1 + x / 2))

:((((0 + (1 * 1) * x) + (1x) * 1) * 1 - (x * x) * 0) / (1 * 1) + (1 * 2 - x * 0) / (2 * 2))

In [67]:
autodiff(:(x + x*x*x))

:(1 + (((0 + ((1 * 1) * x) * x) + ((1x) * 1) * x) + ((1x) * x) * 1))

In [35]:
f(x) = x * x + x - 10

f (generic function with 1 method)

In [10]:
# Zadanie dodatkowe the fun way
using DualNumbers
function autodiffFun(f) 
    g(x) = dualpart(f(Dual(x,1)))
end    

autodiffFun (generic function with 1 method)

In [12]:
f(x) = x*x
g = autodiffFun(f)
g(2)

4